# Conversor de Radiação (Rs → n)

Este notebook converte **radiação solar global diária** `Rs` (MJ/m²/dia) em **horas de sol** `n` (h),
usando a relação de **Angström–Prescott** e cálculos astronômicos diários (FAO-56).

## Como usar (bem simples)
1. Edite apenas os campos **INPUT_DIR** e **OUTPUT_DIR** na célula **Configurações**.
2. Rode o notebook do início ao fim (**Kernel → Restart & Run All**).
3. Os arquivos convertidos serão salvos mantendo a mesma estrutura de pastas do diretório de entrada.

## Formato de entrada (CSV, separador ";")
`dia;mes;ano;valor`  → onde `valor` = Rs (MJ/m²/dia)

## Nome do arquivo (precisa conter latitude/longitude)
Exemplo: `ACCESS-ESM1-5-rss-ssp245_-25.125_-49.125.csv`

> Observação: a latitude/longitude é lida do **nome do arquivo** para calcular N e Ra.


In [ ]:
# ===============================
# CONFIGURAÇÕES (EDITE AQUI)
# ===============================

# Pasta raiz de entrada (vai procurar .csv em todas as subpastas)
INPUT_DIR  = r"E:\CLIMBRA_SSP585\rss_input"

# Pasta raiz de saída (vai criar subpastas iguais às de entrada)
OUTPUT_DIR = r"E:\CLIMBRA_SSP585\rss_output"

# Coeficientes de Angström–Prescott (FAO-56: valores genéricos)
A_ANGSTROM = 0.25
B_ANGSTROM = 0.50

# Se True, limita n entre 0 e N (recomendado)
CLIP_N = True

# Sufixo no nome do arquivo de saída
OUTPUT_SUFFIX = "_n"

# Regex para extrair lat/lon do nome do arquivo:
# ..._-25.125_-49.125.csv
LATLON_REGEX = r"_(?P<lat>-?\d+(?:\.\d+)?)_(?P<lon>-?\d+(?:\.\d+)?)\.csv$"


In [ ]:
import re
from pathlib import Path

import numpy as np
import pandas as pd


In [ ]:
# ======================================================
# FUNÇÕES ASTRONÔMICAS (FAO-56)
# ======================================================

def solar_declination(J):
    """Declinação solar (radianos) – Allen et al. (1998) FAO-56."""
    return 0.409 * np.sin((2.0 * np.pi / 365.0) * J - 1.39)

def inverse_relative_distance_earth_sun(J):
    """Distância relativa Terra–Sol (adimensional)."""
    return 1.0 + 0.033 * np.cos((2.0 * np.pi / 365.0) * J)

def sunset_hour_angle(phi_rad, delta):
    """Ângulo horário ao pôr do sol (radianos)."""
    x = -np.tan(phi_rad) * np.tan(delta)
    x = np.clip(x, -1.0, 1.0)
    return np.arccos(x)

def day_length_hours(lat_deg, J):
    """Duração máxima do dia N (h), para latitude (graus) e dia juliano J."""
    phi = np.radians(lat_deg)
    delta = solar_declination(J)
    omega_s = sunset_hour_angle(phi, delta)
    return 24.0 / np.pi * omega_s

def extraterrestrial_radiation_MJm2d(lat_deg, J):
    """
    Radiação extraterrestre diária Ra (MJ/m²/dia) – FAO-56.

    Ra = (24*60/pi) * Gsc * dr * [ωs*sinφ*sinδ + cosφ*cosδ*sinωs]
    Gsc = 0.0820 MJ·m⁻²·min⁻¹
    """
    Gsc = 0.0820
    dr = inverse_relative_distance_earth_sun(J)
    phi = np.radians(lat_deg)
    delta = solar_declination(J)
    omega_s = sunset_hour_angle(phi, delta)

    Ra = (24.0 * 60.0 / np.pi) * Gsc * dr * (
        omega_s * np.sin(phi) * np.sin(delta)
        + np.cos(phi) * np.cos(delta) * np.sin(omega_s)
    )
    return np.maximum(Ra, 0.0)

# ======================================================
# SUPORTE (leitura, lat/lon, processamento)
# ======================================================

def parse_lat_lon_from_filename(filename: str):
    """Extrai latitude/longitude do nome do arquivo usando LATLON_REGEX."""
    m = re.search(LATLON_REGEX, filename)
    if not m:
        raise ValueError(f"Não foi possível extrair lat/lon do nome: {filename}")
    lat = float(m.group("lat"))
    lon = float(m.group("lon"))  # lon não é usada no cálculo atual, mas é extraída por completude
    return lat, lon

def process_file(path_in: Path, base_in: Path, base_out: Path):
    """
    Processa um CSV:
      - lê dia/mes/ano/valor (Rs)
      - calcula J, N, Ra
      - calcula n (horas de sol) pela Angström–Prescott
      - salva CSV preservando a estrutura de pastas
    """
    print(f"Processando: {path_in}")

    df = pd.read_csv(path_in, sep=";")
    for col in ["dia", "mes", "ano", "valor"]:
        if col not in df.columns:
            raise KeyError(f"Coluna obrigatória '{col}' não encontrada em {path_in.name}")

    df["data"] = pd.to_datetime(
        dict(year=df["ano"], month=df["mes"], day=df["dia"]),
        errors="coerce",
    )
    df = df.dropna(subset=["data"]).copy()

    J = df["data"].dt.dayofyear.astype(int)

    lat, _ = parse_lat_lon_from_filename(path_in.name)

    Rs = df["valor"].astype(float)

    N_h   = day_length_hours(lat, J)
    Ra_MJ = extraterrestrial_radiation_MJm2d(lat, J)

    # Proteção para Ra=0
    Ra_safe = np.where(Ra_MJ > 0, Ra_MJ, np.nan)
    ratio = Rs / Ra_safe

    # Angström–Prescott: Rs/Ra = a + b * (n/N)
    # => n = N * ((Rs/Ra) - a) / b
    n_h = N_h * ((ratio - A_ANGSTROM) / B_ANGSTROM)

    n_h_clip = np.clip(n_h, 0.0, N_h) if CLIP_N else n_h

    df_out = df.copy()
    df_out["Ra_MJm2d"]    = Ra_MJ
    df_out["N_h"]         = N_h
    df_out["Rs_MJm2d"]    = Rs
    df_out["Rs_Ra_ratio"] = ratio
    df_out["n_h"]         = n_h
    df_out["n_h_clip"]    = n_h_clip

    rel_parent = path_in.relative_to(base_in).parent
    out_dir = base_out / rel_parent
    out_dir.mkdir(parents=True, exist_ok=True)

    out_name = path_in.stem + OUTPUT_SUFFIX + path_in.suffix
    path_out = out_dir / out_name

    df_out.to_csv(path_out, sep=";", index=False)
    print(f"  -> Salvo em: {path_out}")

def run_conversion():
    """Roda a conversão em todos os CSVs dentro de INPUT_DIR."""
    base_in = Path(INPUT_DIR)
    base_out = Path(OUTPUT_DIR)
    base_out.mkdir(parents=True, exist_ok=True)

    files = sorted(base_in.rglob("*.csv"))
    if not files:
        print(f"Nenhum arquivo .csv encontrado em {base_in}")
        return

    print(f"Encontrados {len(files)} arquivo(s) para processar.\n")

    for i, f in enumerate(files, start=1):
        try:
            print(f"[{i}/{len(files)}]")
            process_file(f, base_in, base_out)
        except Exception as e:
            print(f"ERRO ao processar {f}: {e}\n")


In [ ]:
# ===============================
# EXECUTAR CONVERSÃO
# ===============================
run_conversion()
